In [25]:
import cv2

In [26]:
def extract_frames(video_loc, saving_frames_loc, terminal_text):
    """
    Extracts frames from a video
    :param video_loc: Path to input video
    :param saving_frames_loc: Path to output frames, must be a folder
    """

    cam = cv2.VideoCapture(video_loc)
    frame_num = 0

    while (True):
        ret, frame = cam.read()
        if ret:
            zero_num = 10
            tmp = frame_num
            while tmp > 0:
                tmp = int(tmp / 10)
                zero_num -= 1
            zeros = ""
            for i in range(zero_num):
                zeros = str(0) + zeros
            name = saving_frames_loc + zeros + str(frame_num) + '.jpg'
            cv2.imwrite(name, frame)
            frame_num += 1
        else:
            break

    cam.release()
    cv2.destroyAllWindows()